In [1]:
#import libraries
import pandas as pd
import numpy as np
import openpyxl as op
from google.colab import files
from openpyxl.utils.dataframe import dataframe_to_rows

#create upload link
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
#copy json file in a directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#permission to access kaggle data
!chmod 600 ~/.kaggle/kaggle.json

#Download kaggle data
!kaggle datasets download -d wendykan/lending-club-loan-data

#Unzip kaggle data
from zipfile import ZipFile
file_name = "lending-club-loan-data.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Unzip Done')

#Craete DataFrame with all data
data = pd.read_csv('loan.csv')
total_rows_Data=len(data.axes[0])
print('Total Rows in Data:', total_rows_Data)

100% 720M/720M [00:05<00:00, 113MB/s]
100% 720M/720M [00:05<00:00, 136MB/s]
Unzip Done


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Total Rows in Data: 2260668


In [ ]:
#Create Working Dataframe
df = data.loc[:, ['annual_inc', 'loan_amnt', 'annual_inc_joint', 'issue_d', 'grade', 'loan_status', 'dti', 'dti_joint', 'acc_now_delinq', 'inq_last_12m', 'inq_fi', 'hardship_flag', 'collections_12_mths_ex_med']]
total_rows = len(df.axes[0])
print('Total Rows in DataFrame',total_rows)

#Replacing NaN value with 0
df['annual_inc'] = df['annual_inc'].replace(np.nan, 1)
df['annual_inc'] = df['annual_inc'].replace(0, 1)
df['annual_inc_joint'] = df['annual_inc_joint'].replace(np.nan, 0)
df['dti'] = df['dti'].replace(np.nan, 400000000)
df['dti_joint'] = df['dti_joint'].replace(np.nan, 0)
df['acc_now_delinq'] = df['acc_now_delinq'].replace(np.nan, 0)
df['inq_last_12m'] = df['inq_last_12m'].replace(np.nan, 1000)
df['inq_fi'] = df['inq_fi'].replace(np.nan, 1000)
df['collections_12_mths_ex_med'] = df['collections_12_mths_ex_med'].replace(np.nan, 0)

#Finding Total number of rows after data cleaning
total_rows = len(df.axes[0])
print('Total Rows in DataFrame',total_rows)

#Calculate Total income, dti and inq
sumColumn = df['annual_inc'] + df['annual_inc_joint']
df['total_inc'] = sumColumn

#Rounded real values to the nearest whole dollar
#df["total_inc"] = df["total_inc"].astype(int)

df['debt'] = df['dti'] * df['annual_inc']
df['debt_joint'] = df['dti_joint'] * df['annual_inc_joint']

df['total_dti'] = (df['debt'] + df['debt_joint']) / (df['annual_inc'] + df['annual_inc_joint'])

sumColumn2 = df['inq_last_12m'] + df['inq_fi']
df['total_inq'] = sumColumn2

#Generate Bands according to total_dti
dti_band = [-2, 10, 15, 20, 25, 100, 65000]
dti_bands_name =[1, 2, 3, 4, 5, 6]
df['dti_band'] = pd.cut(df['total_dti'], dti_band, labels = dti_bands_name)

#Generate Bands according to total_inquiry
inq_band = [-1, 0, 1, 2, 4, 100, 25000]
inq_bands_name =[1, 2, 3, 4, 5, 6]  
df['inq_band'] = pd.cut(df['total_inq'], inq_band, labels = inq_bands_name)

#Generate Band according to Total income
inc_bands = [-1, 1, 45000, 60000, 80000, 110000, 1100000000]
inc_bands_name =[1, 2, 3, 4, 5, 6]
df['inc_band'] = pd.cut(df['total_inc'], inc_bands, labels = inc_bands_name)

#Create bad indicator
#IGP - Including Grace Period
#EGP - Excluding Grace Period
df['bad_flag_IGP'] = df['loan_status']
df['bad_flag_IGP'].replace(to_replace = ['Does not meet the credit policy. Status:Charged Off', 'Charged Off', 'In Grace Period',  'Default', 'Late (16-30 days)', 'Late (31-120 days)', 'Current', 'Fully Paid', 'Does not meet the credit policy. Status:Fully Paid'],
                                value = [1, 1, 1, 1, 1, 1, 0, 0, 0],
                              inplace = True)

df['bad_flag_EGP'] = df['loan_status']
df['bad_flag_EGP'].replace(to_replace = ['Does not meet the credit policy. Status:Charged Off', 'Charged Off', 'In Grace Period',  'Default', 'Late (16-30 days)', 'Late (31-120 days)', 'Current', 'Fully Paid', 'Does not meet the credit policy. Status:Fully Paid'],
                                value = [1, 1, 0, 1, 1, 1, 0, 0, 0],
                              inplace = True)

#Create buraeu bad indicator
df['hardship_flag_bin'] = df['hardship_flag']
df['hardship_flag_bin'].replace(to_replace = ['Y', 'N'],
                                value = [1, 0],
                              inplace = True)

df['delinq_flag'] = df['acc_now_delinq']
df['delinq_flag'].replace(to_replace = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                                value = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                              inplace = True)

df['collection_flag'] = df['collections_12_mths_ex_med']
df['collection_flag'].replace(to_replace = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                                value = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                              inplace = True)

df['bureau_bad'] = df['hardship_flag_bin'] + df['delinq_flag'] + df['collection_flag']

df['bureau_bad'].replace(to_replace = [0, 1, 2, 3],
                                value = [0, 1, 1, 1],
                              inplace = True)

#OA - Over All
df['OA_bad_acc_IGP'] = df['hardship_flag_bin'] + df['delinq_flag'] + df['collection_flag'] + df['bad_flag_IGP']

df['OA_bad_acc_IGP'].replace(to_replace = [0, 1, 2, 3, 4],
                            value = [0, 1, 1, 1, 1],
                              inplace = True)

#OA - Over All
df['OA_bad_acc_EGP'] = df['hardship_flag_bin'] + df['delinq_flag'] + df['collection_flag'] + df['bad_flag_EGP']

df['OA_bad_acc_EGP'].replace(to_replace = [0, 1, 2, 3, 4],
                            value = [0, 1, 1, 1, 1],
                              inplace = True)

df.head()

Total Rows in DataFrame 2260668
Total Rows in DataFrame 2260668


,annual_inc,loan_amnt,annual_inc_joint,issue_d,grade,loan_status,dti,dti_joint,acc_now_delinq,inq_last_12m,inq_fi,hardship_flag,collections_12_mths_ex_med,total_inc,debt,debt_joint,total_dti,total_inq,dti_band,inq_band,inc_band,bad_flag_IGP,bad_flag_EGP,hardship_flag_bin,delinq_flag,collection_flag,bureau_bad,OA_bad_acc_IGP,OA_bad_acc_EGP
0,55000.0,2500,0.0,Dec-2018,C,Current,18.24,0.0,0.0,2.0,1.0,N,0.0,55000.0,1003200.0,0.0,18.24,3.0,3,4,3,0,0,0,0.0,0.0,0.0,0.0,0.0
1,90000.0,30000,0.0,Dec-2018,D,Current,26.52,0.0,0.0,2.0,2.0,N,0.0,90000.0,2386800.0,0.0,26.52,4.0,5,4,5,0,0,0,0.0,0.0,0.0,0.0,0.0
2,59280.0,5000,0.0,Dec-2018,D,Current,10.51,0.0,0.0,0.0,1.0,N,0.0,59280.0,623032.8,0.0,10.51,1.0,2,2,3,0,0,0,0.0,0.0,0.0,0.0,0.0
3,92000.0,4000,0.0,Dec-2018,D,Current,16.74,0.0,0.0,3.0,2.0,N,0.0,92000.0,1540080.0,0.0,16.74,5.0,3,5,5,0,0,0,0.0,0.0,0.0,0.0,0.0
4,57250.0,30000,0.0,Dec-2018,C,Current,26.35,0.0,0.0,0.0,1.0,N,0.0,57250.0,1508537.5,0.0,26.35,1.0,5,2,3,0,0,0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.loc[1:1605,['annual_inc', 'annual_inc_joint', 'dti', 'dti_joint', 'debt', 'debt_joint', 'total_dti' ]]


In [ ]:
values, count = np.unique(df['inq_band'], return_counts=True)

dfTemp = pd.DataFrame()
dfTemp['Values'] = values
dfTemp['Count'] = count

dfTemp.tail(10)
# df[df.dti == 20000].dti

,Values,Count
0,1,301273
1,2,263759
2,3,220194
3,4,285365
4,5,323947
5,6,866130


In [ ]:
df.isna().sum()

**Loan Status of different demographics**

In [ ]:
demographic = 'grade'

x = df.groupby(
   [demographic, 'loan_status']
).agg( 
    {
         'loan_status': "count",
    }
)

filename = "temp.csv"
x.to_csv(filename)
df1 = pd.read_csv('temp.csv', low_memory=False)

df1 = df1.pivot(index = demographic, columns = 'loan_status', values = 'loan_status.1')

df1 = df1.replace(np.nan, 0)

sumColumn3 = df1['Charged Off'] + df1['Does not meet the credit policy. Status:Charged Off']
df1['Charged Off1'] = sumColumn3

sumColumn4 = df1['Does not meet the credit policy. Status:Fully Paid'] + df1['Fully Paid']
df1['Fully Paid1'] = sumColumn4

sumColumn6 = df1['Default'] + df1['Late (16-30 days)'] + df1['Late (31-120 days)'] + df1['In Grace Period']
df1['Deli_inc_grc_prd'] = sumColumn6

sumColumn5 = df1['Default'] + df1['Late (16-30 days)'] + df1['Late (31-120 days)']
df1['Deli_exc_grc_prd'] = sumColumn5

df1  = df1.drop(['Default', 'In Grace Period', 'Late (16-30 days)',  'Late (31-120 days)', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off', 'Charged Off', 'Fully Paid'], axis=1)

#Finding Total number of rows after data cleaning
total_rows = len(df.axes[0])
print('Total Rows in DataFrame',total_rows)

df1.head(10)

Total Rows in DataFrame 2260668


loan_status,Current,Charged Off1,Fully Paid1,Deli_inc_grc_prd,Deli_exc_grc_prd
grade,,,,,
A,204435.0,13774.0,212567.0,2251.0,1583.0
B,275455.0,51162.0,329344.0,7596.0,5588.0
C,267326.0,83410.0,287147.0,12170.0,8976.0
D,120666.0,59638.0,136333.0,7787.0,5822.0
E,40174.0,35522.0,56584.0,3359.0,2565.0
F,9006.0,14356.0,17374.0,1064.0,834.0
G,2633.0,4554.0,4591.0,390.0,297.0


**Bureau Bad of different Demographics**

In [ ]:
demographic = 'grade'

x = df.groupby(
   [demographic, 'bureau_bad']
).agg( 
    {
         'bureau_bad': "count",
    }
)

filename = "temp.csv"
x.to_csv(filename)
df1 = pd.read_csv('temp.csv', low_memory=False)

df1 = df1.pivot(index = demographic, columns = 'bureau_bad', values = 'bureau_bad.1')

df1 = df1.replace(np.nan, 0)
df1.head(10)

bureau_bad,0.0,1.0
grade,,
A,428576,4451
B,649692,13865
C,634407,15646
D,316315,8109
E,132318,3321
F,40808,992
G,11870,298


**Over All Bureau Bad of different Demographics including Grace Period**

In [ ]:
demographic = 'inq_band'

x = df.groupby(
   [demographic, 'OA_bad_acc_IGP']
).agg( 
    {
         'OA_bad_acc_IGP': "count",
    }
)

filename = "temp.csv"
x.to_csv(filename)
df1 = pd.read_csv('temp.csv', low_memory=False)

df1 = df1.pivot(index = demographic, columns = 'OA_bad_acc_IGP', values = 'OA_bad_acc_IGP.1')

df1 = df1.replace(np.nan, 0)
df1.head(10)

OA_bad_acc_IGP,0.0,1.0
inq_band,,
1,272917,28356
2,235053,28706
3,193928,26266
4,247460,37905
5,273191,50756
6,702652,163478


**Over All Bureau Bad of different Demographics Excluding Grace Period**

In [ ]:
demographic = 'inq_band'

x = df.groupby(
   [demographic, 'OA_bad_acc_EGP']
).agg( 
    {
         'OA_bad_acc_EGP': "count",
    }
)

filename = "temp.csv"
x.to_csv(filename)
df1 = pd.read_csv('temp.csv', low_memory=False) 

df1 = df1.pivot(index = demographic, columns = 'OA_bad_acc_EGP', values = 'OA_bad_acc_EGP.1')

df1 = df1.replace(np.nan, 0)
df1.head(10)

OA_bad_acc_EGP,0.0,1.0
inq_band,,
1,274349,26924
2,236423,27336
3,195171,25023
4,249135,36230
5,275325,48622
6,703520,162610


In [ ]:
x = df.groupby(
   ['dti_band', 'inq_band', 'inc_band', 'grade', 'loan_status']
).agg( 
    {
         'loan_status': "count",
    }
)

filename = "temp.csv"
x.to_csv(filename)
df1 = pd.read_csv('temp.csv', low_memory=False)

#df1 = df1.pivot(index = ['dti_band', 'inq_band', 'inc_band', 'grade'], columns = 'loan_status', values = 'loan_status.1')

df1 = df1.replace(np.nan, 0)
pd.set_option('display.max_columns', 100)
df1.head()

,dti_band,inq_band,inc_band,grade,loan_status,loan_status.1
0,1,1,1,A,Charged Off,507
1,1,1,1,A,Current,4040
2,1,1,1,A,Default,0
3,1,1,1,A,Does not meet the credit policy. Status:Charge...,3
4,1,1,1,A,Does not meet the credit policy. Status:Fully ...,17
